# Column Based Redshift

### Introduction

As we know, redshift is used to store our data for our analytical database.  And as we've discussed, our analytical queries are normally performed by querying the data along a column, as opposed to selecting a single row.  Because of this, redshift provides column based storage.  In this lesson, we'll describe what this means and see some of the benefits that this provides. 

### What's column based storage

First, let's remember the difference between our typical OLTP queries and our OLAP queries.  

* OLTP queries

With our OLTP queries, we are performing queries for a web application, and these queries often involve a single entity.  For example, we may need to return information about a user, or a stock, or a restaurant.   When doing so, we want to view many of the attributes in that row - like multiple details about a restaurant, or a user. 

* OLAP queries

With our analytical database.  We may not need to know everything about a single entity, but rather we care about looking at some select attribute -- say the prices and ratings of restaurants, across many different entities.

How do databases account for these differences?  Well, with our row based OLTP queries, there is row based storage.  

* Row based storage

Let's say that we would like to store the following movies.


<img src="./movie-table.png" width="70%">

Now the way that a row-based database like postgres will store this on disk, is sequentially, with the entries of one full row and then the next full row.

<img src="./row-down-storage.png" width="20%">

So we can see that first comes the contents of Pulp Fiction, and then comes the contents of Titanic and then the contents of Avatar.  That the row is stored in sequence allows for an entire row to more quickly read from the disk.

* Column based storage

If row based storage stores the entries of a row sequentially, then, as we would suspect, column based storage stores the entries of our columns sequentially.  We can see this below.

<img src="./total_column.png" width="70%">

So the diagram above is trying to illustrate that each of the columns are stored separately.  But we still have a way of keeping the values of a single row aligned with one another.

In redshift, each one of the column blocks above is called a data block.  And each data block can store one million entries.  So why is this useful?  Well, if we have a query that is say `SELECT year from movies;`, then redshift will not bother with loading up any of the other columns, it will only select the values from the year data block.  This allows the query to perform faster, and takes less space in memory.

So we can see that the way that redshift stores data is designed for our column-based analytical queries.

### Additional Optimizations: Compression, and Zone Maps

There are other features that redshift can take advantage of with column based storage.  The first is compression.  Because all of the values in a data block are generally of the same type, redshift automatically encodes each column in a suitable way, to compress storage.

> The below, is an example of how redshift automatically encodes columns.

<img src="./analyze_compression.png" width="60%">

* Zone maps

Redshift offers another important feature to allow for fast retrieval of data, and that's keeping track of what data is stored in which specific data blocks.  Let's take another look at our data blocks.

<img src="./total_column.png" width="70%">

If we performed a query of say `SELECT title FROM movies WHERE year = 1994`, we may have to search for the matching records through a movies table that is millions of records long, stored across multiple data blocks.  To avoid searching through every single data block, Redshift keeps track of minimum and maximum of a data block in memory.  This way it knows if the value is in the data block's range.  

So in the above example, redshift would not have to search through a data block for a year of 1994, if the minimum and maximum movie years in that data block were 1960 - 1981.  Note that redshift will keep track of the minimum and maximum values even if the data is not sorted, but we can optimize the benefit of zone maps through sorting certain columns (often datetimes).

### Summary

In this lesson, we learned about the difference betweeen row based and column based storage and how redshift improves the performance of queries through column based storage.  First, we saw that through storing a single column of data per data block, redshift can only load up specifically queried columns, and avoid reading extra data into memory.  Then we saw that because data stored in together is of the same type, it can compress the data to save disk space.  Finally, we saw that redshift, stores in memory the minimum and max values in a data block -- via zone maps -- so that it does not have to scan each data block when performing a query.

### Resources

[Redshift Advanced Design](https://aws.amazon.com/blogs/big-data/amazon-redshift-engineerings-advanced-table-design-playbook-compound-and-interleaved-sort-keys/)

[Redshift deep dive video](https://youtu.be/iuQgZDs-W7A?t=578)

[Redshift sorting and zone maps blog](https://dev.to/ronsoak/the-r-a-g-redshift-analyst-guide-sorting-zone-maps-51cf)